In [1]:
# ========================================
# PASSO DE VALIDAÇÃO DO AMBIENTE
# ========================================

import os
from dotenv import load_dotenv
from openai import AzureOpenAI

print("--- Iniciando Validação ---")

# 1. Carregar variáveis do arquivo .env
# (Garante que seu arquivo .env está sendo lido)
load_dotenv()

# 2. Ler as variáveis de ambiente
# (Garante que os NOMES das variáveis no .env estão corretos)
endpoint = os.getenv("AZURE_MODEL_ENDPOINT")
deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
api_key = os.getenv("AZURE_AI_MODEL_API_KEY")
api_version = "2024-12-01-preview" # Versão padrão da API

# 3. Verificar se as variáveis foram carregadas
# (Se alguma aparecer como "None", verifique seu .env)
print(f"Endpoint Carregado: {endpoint}")
print(f"Deployment Carregado: {deployment}")
if api_key:
    print(f"Chave Carregada: {api_key[:4]}...{'*' * 10}")
else:
    print("ERRO: Chave API (api_key) não encontrada. Verifique seu .env.")

# 4. Tentar a conexão (O teste final)
if endpoint and deployment and api_key:
    try:
        print("\n--- Testando Conexão com Azure OpenAI ---")
        client = AzureOpenAI(
            api_version=api_version,
            azure_endpoint=endpoint,
            api_key=api_key,
        )

        response = client.chat.completions.create(
            messages=[{"role": "user", "content": "Olá, isso é um teste. Responda 'OK'."}],
            model=deployment
        )
        
        print("\n========================================")
        print("✅ SUCESSO! Ambiente Validado!")
        print(f"Resposta da IA: {response.choices[0].message.content}")
        print("========================================")

    except Exception as e:
        print(f"\n========================================")
        print(f"❌ ERRO NA VALIDAÇÃO: {e}")
        print("========================================")
        print("Por favor, verifique seus valores no .env (Endpoint, Deployment Name, API Key).")
else:
    print("\nERRO: Uma ou mais variáveis de ambiente não foram carregadas. Verifique o arquivo .env.")

--- Iniciando Validação ---
Endpoint Carregado: https://amand-mhb8nq42-swedencentral.cognitiveservices.azure.com/
Deployment Carregado: gpt-4.1
Chave Carregada: F19q...**********

--- Testando Conexão com Azure OpenAI ---

✅ SUCESSO! Ambiente Validado!
Resposta da IA: OK


In [2]:
# ========================================
# PASSO 1: Importações e Definição da "Tool"
# ========================================
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Imports do Agent Framework
from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import ChatAgent

print("Importações concluídas.")

# Carregar variáveis do .env
load_dotenv()
endpoint_model = os.getenv("AZURE_MODEL_ENDPOINT")
deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
model_key = os.getenv("AZURE_AI_MODEL_API_KEY")

print("Variáveis de ambiente carregadas.")

# ========================================
# SUA "TOOL" PERSONALIZADA (40 PONTOS!)
# ========================================

def calcular_custo_empregabilidade(numero_pessoas: int, meses_contrato: int, salario_minimo: float) -> str:
    """
    Calcula o custo total de um programa de empregabilidade verde.
    O agente DEVE ter esta informação para usar a ferramenta.

    Args:
        numero_pessoas (int): O número total de pessoas a serem contratadas.
        meses_contrato (int): O número de meses do contrato.
        salario_minimo (float): O valor do salário mínimo (ex: 1518.00).
    
    Returns:
        str: Uma string formatada com a mensagem do custo total em Reais (R$).
    """
    
    # Esta é a sua lógica de negócio (a "1 ação funcional")
    custo_total = numero_pessoas * meses_contrato * salario_minimo
    
    # Formata a resposta
    resultado = f"O custo estimado para contratar {numero_pessoas} pessoas por {meses_contrato} meses, com salário de R${salario_minimo:,.2f}, é de R$ {custo_total:,.2f}."
    
    print(f"[Debug da Tool]: A função 'calcular_custo_empregabilidade' foi chamada. Resultado: {resultado}")
    return resultado

print("✅ Tool 'calcular_custo_empregabilidade' definida com sucesso!")

Importações concluídas.
Variáveis de ambiente carregadas.
✅ Tool 'calcular_custo_empregabilidade' definida com sucesso!


In [9]:
# ========================================
# PASSO 2: Criando o "Agente ESG" (Versão Limpa)
# ========================================

async def criar_agente_esg():
    """
    Cria um agente especialista em ESG com a calculadora de custo.
    """
    # 1. A autenticação é feita aqui, direto no cliente
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-02-15-preview", 
    )
    
    # 2. NÃO precisamos mais do 'credential = AzureCliCredential()'
    
    esg_agent = ChatAgent(
        chat_client=chat_client,
        name="agente_esg_calculadora",
        instructions=(
            "Você é um assistente especialista em ESG e políticas públicas, focado em dados do Brasil. "
            "Seu trabalho é ajudar gestores públicos a estimar custos de programas de empregabilidade verde. "
            "Quando o usuário perguntar 'quanto custa' (ou 'qual o custo', 'estimativa', 'calcular'), "
            "você DEVE usar a ferramenta 'calcular_custo_empregabilidade'. "
            "Para isso, você precisa dos 3 valores: numero_pessoas, meses_contrato e salario_minimo. "
            "Se o usuário não fornecer os 3 valores, pergunte educadamente por eles. "
            "Responda sempre em Reais (R$). Não invente valores."
        ),
        tools=[calcular_custo_empregabilidade] 
    )

    # 3. Retorne apenas o agente
    return esg_agent

print("✅ Função 'criar_agente_esg' (Versão Limpa) definida com sucesso!")

✅ Função 'criar_agente_esg' (Versão Limpa) definida com sucesso!


In [10]:
# ========================================
# PASSO 3: Executando o Agente ESG! (Versão Limpa)
# ========================================

pergunta = "Olá! Quanto custaria contratar 1.500 pessoas por 6 meses, com o salário mínimo de R$ 1.518,00?"

async def rodar_agente_esg(pergunta_do_usuario):
    
    # 1. Agora só recebemos o agente
    esg_agent = await criar_agente_esg()

    print(f"Enviando pergunta para o Agente ESG: '{pergunta_do_usuario}'")
    print("...aguardando resposta da IA...")
    
    resultado = await esg_agent.run(pergunta_do_usuario)
    
    print("\n=== Resposta do Agente ESG ===")
    print(resultado)
    
    # 4. NÃO precisamos mais fechar a credencial
    print("\n--- Processo Concluído ---")

# =C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C=
# ESTE É O MOMENTO DE TIRAR O PRINT (SCREENSHOT) PARA O SEU README! 📸
# =C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C=
await rodar_agente_esg(pergunta)

Enviando pergunta para o Agente ESG: 'Olá! Quanto custaria contratar 1.500 pessoas por 6 meses, com o salário mínimo de R$ 1.518,00?'
...aguardando resposta da IA...
[Debug da Tool]: A função 'calcular_custo_empregabilidade' foi chamada. Resultado: O custo estimado para contratar 1500 pessoas por 6 meses, com salário de R$1,518.00, é de R$ 13,662,000.00.

=== Resposta do Agente ESG ===
O custo estimado para contratar 1.500 pessoas por 6 meses, pagando o salário mínimo de R$ 1.518,00, é de R$ 13.662.000,00. Se precisar de uma estimativa para outro cenário, posso calcular para você!

--- Processo Concluído ---
